In [ ]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

get_weather_tool = {
    "type": "function",
    "name": "search",
    "description":  "Search the Weather database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description":  "Search query text to look up in the Weather..."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [14]:
from fastmcp import Client

import weather_server

# async def main():
#     async with Client(weather_server.mcp) as mcp_client:
#         temp = await mcp_client.call("get_weather", {"city": "Berlin"})
#         print(f"Temperatura em Berlin: {temp}°C")

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = mcp_client.list_tools()
        
        for tool in tools:
            print(f"\nNome: {tool['name']}")
            print(f"Descrição: {tool['description']}")
            print("Parâmetros:")
            for param, details in tool['parameters']['properties'].items():
                print(f"  - {param}: {details['description']}")

In [ ]:
import chat_assistant

import requests
from minsearch import AppendableIndex

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

tools = chat_assistant.Tools()
tools.add_tool(search, get_weather_tool)

tools.get_tools()

from openai import OpenAI
client = OpenAI()

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

chat.run()